<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-core asknews langgraph
%pip install --quiet -U "langchain-community>=0.2.16"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8

In [3]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools import JinaSearch
from dotenv import load_dotenv
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"
SERPER_API_KEY = "d8e815ef6caa94dbef7b977a0ea7d505b43a5a06"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama-3.1-8b-instant", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Pinecone and vector store
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

# Initialize search tool
asknews_tool = AskNewsSearch(max_results=5)
google_serper = GoogleSerperAPIWrapper()
jina_search = JinaSearch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=5)
    return "\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(results)])

def search_wrapper(search_func, query: str) -> str:
    results = search_func({"query": query})
    formatted_results = []
    for i, result in enumerate(results, 1):
        formatted_results.append(f"Result {i}:\nTitle: {result.get('title', 'No title')}\nSnippet: {result.get('snippet', 'No snippet')}\nURL: {result.get('link', 'No link')}")
    return "\n\n".join(formatted_results)

def execute_searches(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    searches = [
        ("Vector Search", vector_search),
        ("AskNews Search", lambda q: asknews_tool.invoke({"query": q})),
        ("Google Serper Search", lambda q: search_wrapper(google_serper.run, q)),
        ("Jina Search", lambda q: search_wrapper(jina_search.invoke, q))
    ]

    for name, func in searches:
        try:
            result = func(query)
            state["messages"].append({"role": "tool", "content": f"{name} Result: {result}"})
        except Exception as e:
            state["messages"].append({"role": "tool", "content": f"{name} Error: {str(e)}"})

    return state

In [6]:
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    prompt = ChatPromptTemplate.from_messages([(
        "system", """You are an advanced AI specializing in cybersecurity and intelligence analysis. Your task is to provide highly relevant, actionable, and up-to-date insights based on the latest cyber incidents. Follow these guidelines:

1. Analyze all search results thoroughly, prioritizing the most recent and relevant information.
2. Focus on information from reputable sources, official reports, and verified cybersecurity platforms.
3. Identify emerging patterns, trends, and potential implications of the latest cyber incidents.
4. Provide a structured response with the following sections:
   a. Key Highlights (bullet points of the most critical findings)
   b. Trends & Implications (numbered list of important trends and their potential impact)
   c. Critical Action Items (prioritized list of recommended actions)
   d. Emerging Concerns (potential future threats or evolving risks)
   e. Research Gaps (areas that require further investigation)

5. Include clear citations for all information, linking to specific search results using [Source X] format.
6. Prioritize information from the last 30 days. If using older sources, clearly state the date.
7. If critical information is missing, identify gaps and suggest additional areas for investigation.
8. Maintain a concise yet comprehensive format, ensuring all key points are covered within a 500-word limit.
9. Use formatting for emphasis: '''bold''' for section headers, and • for bullet points.

Previous conversation: {chat_history}
Human query: {input}
Vector Search Result: {vector_result}
AskNews Search Result: {asknews_result}
Google Serper Search Result: {google_serper_result}
Jina Search Result: {jina_result}

Provide a structured, actionable response based on the latest findings:
"""
    )])

    chain = prompt | llm

    response = chain.invoke({
        "input": state["messages"][-1]["content"],
        "vector_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results."),
        "asknews_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "AskNews" in m["content"]), "No AskNews results."),
        "google_serper_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Google Serper" in m["content"]), "No Google Serper results."),
        "jina_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Jina Search" in m["content"]), "No Jina results."),
        "chat_history": chat_history
    })

    state["messages"].append({"role": "assistant", "content": response.content})
    state["memory"] = {"chat_history": chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {response.content}"}
    return state

# Workflow definition
workflow = StateGraph(AgentState)
workflow.add_node("execute_searches", execute_searches)
workflow.add_node("generate_response", generate_response)
workflow.add_edge("execute_searches", "generate_response")
workflow.add_edge("generate_response", END)
workflow.set_entry_point("execute_searches")
graph = workflow.compile()

def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory or {})
    return graph.invoke(state)

In [7]:
if __name__ == "__main__":
    query = "Latest Cyber Incidents from Blackbasta Ransomware?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot Analysis:")
            print(message["content"])

AI Copilot Analysis:
**Key Highlights**

• Ransomware attacks saw a 14% increase in August, with 450 attacks reported, according to NCC Group's Monthly Threat Pulse [Source 1].
• The number of ransomware double-extortion groups posting to leak sites reached an all-time high in May 2024, with 40 groups actively listing victims, according to Secureworks' State of the Threat 8th Edition report [Source 4].
• Homeland Security Investigations (HSI) has prevented 537 ransomware acts since 2021, with 150 of them occurring between October 2023 and September 30 of this year [Source 5].
• The value of funds taken in hacking activities and ransomware attacks increased by 84% in the first seven months of 2024, rising from $857 million to $1.58 billion [Source 5].
• AI chatbots are being used by criminals to create sophisticated malware, with hackers infiltrating software development tools to write malicious code [Source 5].
• The hacking group GoldenJackal has developed custom USB-focused malware t